In [4]:
import pandas as pd 
import json
import os
import glob
import cv2

In [25]:
def create_yolo_txt_from_json(json_path, txt_dir,data_path):
    with open(json_path, 'r') as json_file:
        json_data = json.load(json_file)

    for filename, data in json_data.items():
        txt_data = []
        regions = data['regions']
        for region in regions:
            shape_attributes = region['shape_attributes']
            region_attributes = region['region_attributes']

            if shape_attributes['name'] == 'ellipse' or shape_attributes['name'] == 'circle':
                cx = int(shape_attributes['cx'])
                cy = int(shape_attributes['cy'])

                if shape_attributes['name'] == 'ellipse':
                    rx = int(shape_attributes['rx'])
                    ry = int(shape_attributes['ry'])
                else:  # shape_attributes['name'] == 'circle'
                    r = int(shape_attributes['r'])
                    rx = ry = r
                    
                img = cv2.imread(data_path+data['filename'])
                wid=img.shape[1]
                hij=img.shape[0]
                
                x1 = int(cx - rx)
                y1 = cy
                x2 = cx - int(rx / 2)
                y2 = cy - int(ry / 2)
                x3 = cx
                y3 = cy - ry
                x4 = cx + int(rx / 2)
                y4 = cy - int(ry / 2)
                x5 = cx + rx
                y5 = cy
                x6 = cx + int(rx / 2)
                y6 = cy + int(ry / 2)
                x7 = cx
                y7 = cy + ry
                x8 = cx - int(rx / 2)
                y8 = cy + int(ry / 2)

                label = region_attributes['cls'][-1:]
                txt_data.append(f'{label} {x1 / wid} {y1 / hij} {x2 / wid} {y2 / hij} {x3 / wid} {y3/hij} {x4 / wid} {y4/ hij} {x5 / wid} {y5/ hij} {x6 / wid} {y6/ hij} {x7 / wid} {y7/ hij} {x8 / wid} {y8/ hij}')

            elif shape_attributes['name'] == 'polyline':
                all_points_x = shape_attributes['all_points_x']
                all_points_y = shape_attributes['all_points_y']
                label = region_attributes['cls'][-1:]
                
                img = cv2.imread(data_path+data['filename'])
                wid=img.shape[1]
                hij=img.shape[0]
                
                # polyline 형태의 데이터를 하나의 라인으로 합침
                txt_data.append(label + ' ' + ' '.join(f'{x / wid} {y / hij}' for x, y in zip(all_points_x, all_points_y)))
#                 txt_data.extend([label] + [f'{x} {y}' for x, y in zip(all_points_x, all_points_y)])
#                 txt_data = ' '.join(txt_data.split())

#                 txt_data.append(label)
#                 for x, y in zip(all_points_x, all_points_y):
#                     txt_data.append(f'{x} {y}')
#                 print(label)
#                 txt_data.append(f'{label} {" ".join(map(str, all_points_x))} {" ".join(map(str, all_points_y))}')

        tmp = data['filename'][:-4]
        txt_file_path = os.path.join(txt_dir, f'{tmp}.txt')
        with open(txt_file_path, 'w') as txt_file:
            txt_file.write('\n'.join(txt_data))

In [ ]:
(1944, 2592, 3)
width = 1920
height = 1080

width = 2592
height = 1944

In [95]:
json_file_path=glob.glob('D:/hm/Projoects/yolo8/data/json/*/*.json')

In [96]:
json_file_path

['D:/hm/Projoects/yolo8/data/json\\test\\via_region_data.json',
 'D:/hm/Projoects/yolo8/data/json\\train\\via_region_data.json',
 'D:/hm/Projoects/yolo8/data/json\\train\\via_region_data_lv2.json',
 'D:/hm/Projoects/yolo8/data/json\\train\\via_region_data_lv3.json',
 'D:/hm/Projoects/yolo8/data/json\\val\\via_region_data.json',
 'D:/hm/Projoects/yolo8/data/json\\val\\via_region_data_lv2.json',
 'D:/hm/Projoects/yolo8/data/json\\val\\via_region_data_lv3.json']

In [5]:
json_file_path_1 = 'D:/hm/Projoects/yolo8/data/json\\train\\via_region_data.json'
json_file_path_2 = 'D:/hm/Projoects/yolo8/data/json\\val\\via_region_data.json'
json_file_path_3 = 'D:/hm/Projoects/yolo8/data/json\\test\\via_region_data.json'

In [69]:
with open(json_file_path_1, 'r', encoding='utf-8') as json_file:
    json_data = json.load(json_file)

In [70]:
json_data

{'20220929_044115.821687_32143.00_17.99.jpg628114': {'filename': '20220929_044115.821687_32143.00_17.99.jpg',
  'size': 628114,
  'regions': [{'shape_attributes': {'name': 'polyline',
     'all_points_x': [1727, 1620, 1520, 1699, 1780, 1772, 1823, 1737],
     'all_points_y': [629, 1105, 1549, 1079, 984, 808, 638, 631]},
    'region_attributes': {'cls': 'lv2'}}],
  'file_attributes': {}},
 '20220905_144347_101190_20.jpg867891': {'filename': '20220905_144347_101190_20.jpg',
  'size': 867891,
  'regions': [{'shape_attributes': {'name': 'ellipse',
     'cx': 1222,
     'cy': 695,
     'rx': 49,
     'ry': 46,
     'theta': 0},
    'region_attributes': {'cls': 'lv3'}},
   {'shape_attributes': {'name': 'ellipse',
     'cx': 1274,
     'cy': 902,
     'rx': 51,
     'ry': 46,
     'theta': 0},
    'region_attributes': {'cls': 'lv3'}},
   {'shape_attributes': {'name': 'polyline',
     'all_points_x': [1281, 1233, 1262, 1342, 1350, 1279],
     'all_points_y': [694, 750, 801, 799, 728, 689]},
  

In [71]:
# from ultralytics.data.converter import convert_coco
## 안댐 
## labe= 3: 맺힌거 2: 흐르는거 

In [26]:
# 사용 예시 
data_path='D:/hm/Projoects/yolo8/data/images\\train/'
txt_output_dir_1 = 'D:/hm/Projoects/yolo8/data/labels\\train/'
create_yolo_txt_from_json(json_file_path_1, txt_output_dir_1,data_path)

In [27]:
# 사용 예시
data_path='D:/hm/Projoects/yolo8/data/images\\val/'
txt_output_dir_2 = 'D:/hm/Projoects/yolo8/data/labels\\val/'
create_yolo_txt_from_json(json_file_path_2, txt_output_dir_2,data_path)

In [10]:
# 사용 예시 json 이상하지만 일단 pass
data_path='D:/hm/Projoects/yolo8/data/images\\test/'

txt_output_dir_3 = 'D:/hm/Projoects/yolo8/data/labels\\test/'
create_yolo_txt_from_json(json_file_path_3, txt_output_dir_3,data_path)

In [21]:
# data_path=r'D:\hm\Projoects\yolo8\data\labels\train'
data_path=r'D:\hm\Projoects\yolo8\data\labels\val/'

In [22]:
def change_class(cls):
    if cls == 2:
        return 0
    elif cls == 3:
        return 1
    else:
        return cls

In [23]:
directory = r'D:\hm\Projoects\yolo8\data\labels\val'
 
for filename in os.listdir(data_path):
    full_path = os.path.join(directory, filename)
    with open(full_path, 'r') as file:
        lines = file.readlines()
    with open(full_path, 'w') as file:
        for line in lines:
            elements = line.strip().split()
            if elements:  # Process only non-empty lines
                new_class = change_class(int(elements[0]))
                elements[0] = str(new_class)
                new_line = ' '.join(elements)
                file.write(new_line + '\n')

    print(f'Classes in the {filename} file have been changed.')

Classes in the 20220725_140343.txt file have been changed.
Classes in the 20220725_140635.txt file have been changed.
Classes in the 20220725_140652.txt file have been changed.
Classes in the 20220905_132955_90476.00_10.99.txt file have been changed.
Classes in the 20220905_134432_183333_0.txt file have been changed.
Classes in the 20220905_134609_157143_22.txt file have been changed.
Classes in the 20220905_134621_69048_21.txt file have been changed.
Classes in the 20220905_134801_167857_17.txt file have been changed.
Classes in the 20220905_135158_95238_6.txt file have been changed.
Classes in the 20220905_135242_150000_15.txt file have been changed.
Classes in the 20220905_135836_51190_21.txt file have been changed.
Classes in the 20220905_140310_98810_7.txt file have been changed.
Classes in the 20220905_140441_119048_13.txt file have been changed.
Classes in the 20220905_140450_171429_18.txt file have been changed.
Classes in the 20220905_140506_159524_19.txt file have been change

In [24]:
import cv2
import os

image_directory = r'D:\hm\Projoects\yolo8\Storage_temp\Data\Image\TM_Leak\20220929'

In [25]:
os.listdir(image_directory)

['20220929_131650',
 '20220929_131708',
 '20220929_131736',
 '20220929_132020',
 '20220929_132104',
 '20220929_132347',
 '20220929_132819',
 '20220929_132913',
 '20220929_133256',
 '20220929_134008',
 '20220929_134058',
 '20220929_134500',
 'CLS']

In [34]:
for i in os.listdir(image_directory):
    tmp=os.path.join(image_directory, i)
    image_files = sorted([f for f in os.listdir(tmp) if f.endswith('.jpg')])
    
    first_image = cv2.imread(os.path.join(tmp, image_files[0]))
    height, width, layers = first_image.shape

    # Define the output video filename and codec
    output_video = 'data/video/'+i+'.avi'
    fourcc = cv2.VideoWriter_fourcc(*'XVID')

    # Create a VideoWriter object
    video = cv2.VideoWriter(output_video, fourcc, 10, (width, height))

    # Loop through the image files and add them to the video
    for image_file in image_files:
        image_path = os.path.join(tmp, image_file)
        frame = cv2.imread(image_path)
        video.write(frame)

    # Release the video writer
    video.release()

    print(f'Video "{output_video}" created successfully.')

Video "data/video/20220929_131650.avi" created successfully.
Video "data/video/20220929_131708.avi" created successfully.
Video "data/video/20220929_131736.avi" created successfully.
Video "data/video/20220929_132020.avi" created successfully.
Video "data/video/20220929_132104.avi" created successfully.
Video "data/video/20220929_132347.avi" created successfully.
Video "data/video/20220929_132819.avi" created successfully.
Video "data/video/20220929_132913.avi" created successfully.
Video "data/video/20220929_133256.avi" created successfully.
Video "data/video/20220929_134008.avi" created successfully.
Video "data/video/20220929_134058.avi" created successfully.
Video "data/video/20220929_134500.avi" created successfully.
